In [8]:
!bash --login -c "poetry update"

Updating dependencies
Resolving dependencies... (9.0s)

No dependencies to install or update


# Option valuation model based on historical prices

This code assumes `aws configure` has been run in the environment of the notebook.

In [1]:
import optionspricing
import boto3
import helpers
import pandas
from dotenv import load_dotenv

load_dotenv()

BINANCE_SYMBOL = 'ETHUSDT'

s3 = boto3.resource('s3')

data_file = helpers.fetch_object(s3, "test-binance-prices-255120844515", f"{BINANCE_SYMBOL}-full.csv.zip")
prices_df = pandas.read_csv(data_file, compression='zip', header=0, index_col="dateTime")
prices_df.index = pandas.to_datetime(prices_df.index)

## Option valuation model: input parameters here

In [2]:
target_period_hours = 2 * 24
strikes_universe_size = 4
CUT_OFF_YEAR_MONTH = (2021, 1)

instrument_code = BINANCE_SYMBOL[:3]

headers = {"Content-Type": "application/json"}
base_url = "https://www.deribit.com/api/v2/public"

trading_model = optionspricing.TradingModel(base_url, headers, instrument_code, target_period_hours)
trading_model.cutoff_year_month(CUT_OFF_YEAR_MONTH)
target_expiry = trading_model.target_expiry
underlying_price = trading_model.underlying_price
remaining_hours = trading_model.remaining_hours

option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size)
simulation = trading_model.simulate_strategy_long_straddle(option_chain_df, strikes_universe_size)
print(simulation)

target expiry: Sat 16 Mar, 08:00 (43 hours left)
current price: 3932.59
-------------------------------
hit ratio: 40%
cost: 0.030 / value: 0.034, gain% = 0.38%
($) cost: 119.94 / value: 135.08, average gain = 15.13
buy put 3900.0
buy call 3950.0
-------------------------------
hit ratio: 38%
cost: 0.026 / value: 0.029, gain% = 0.34%
($) cost: 102.25 / value: 115.76, average gain = 13.51
buy put 3875.0
buy call 3975.0
-------------------------------
hit ratio: 36%
cost: 0.021 / value: 0.025, gain% = 0.37%
($) cost: 84.55 / value: 99.27, average gain = 14.72
buy put 3850.0
buy call 4000.0
-------------------------------
hit ratio: 34%
cost: 0.018 / value: 0.022, gain% = 0.37%
($) cost: 70.79 / value: 85.15, average gain = 14.37
buy put 3825.0
buy call 4025.0


In [3]:
simulation.options_chain

,value_call,value_call_pct,call_bid,call_ask,value_put,value_put_pct,put_bid,put_ask
strike,,,,,,,,
3825.0,160.512579,NaN,0.0325,0.0470,34.584382,0.008789,0.0065,0.0075
3850.0,141.467339,NaN,0.0280,0.0395,40.539142,0.010302,0.0080,0.0090
3875.0,123.566344,NaN,0.0245,0.0305,47.638148,0.012107,0.0100,0.0110
3900.0,107.019053,NaN,0.0220,0.0255,56.090856,0.014255,0.0125,0.0135
3925.0,92.132681,NaN,0.0200,0.0210,66.204484,0.016825,0.0150,0.0160
3950.0,79.296090,0.020152,0.0165,0.0180,78.367893,NaN,0.0180,0.0190
3975.0,68.417843,0.017387,0.0135,0.0150,92.489646,NaN,0.0210,0.0240
4000.0,59.018016,0.014999,0.0115,0.0125,108.089819,NaN,0.0245,0.0280
4025.0,50.842419,0.012921,0.0090,0.0105,124.914223,NaN,0.0285,0.0320


# Sanity checks

In [5]:
size = 8
option_chain_df = trading_model.evaluate(prices_df, strikes_universe_size=size)
put_weights = [0.] * (2 * size + 1)
call_weights = [0.] * (2 * size + 1)
put_weights[size - 5] = 0.
put_weights[size - 2] = 1.
call_weights[size + 2] = 1.
call_weights[size + 5] = 0.

simulation = trading_model.simulate_strategy(option_chain_df, put_weights, call_weights)
print(simulation)

target expiry: Sat 16 Mar, 08:00 (43 hours left)
current price: 3932.59
-------------------------------
hit ratio: 39%
cost: 0.026 / value: 0.029, gain% = 0.39%
($) cost: 100.28 / value: 115.76, average gain = 15.47
buy put 3875.0
buy call 3975.0
